<h2><center>Data Analytics in Finance and Accounting</center></h2>
<h3><center>Case Assignment 1: SE / CRSP Merge</center></h3>
<h3><center>Mark Wahrenburg & Sasan Mansouri Winter Semester 2022-3</center></h3>
<h3><center>Prepared by: Javid Rasulov - 7743063</center></h3>

***

### 1. How many companies have you managed to merge?
> I have managed to merge 205,811 companies.

### 2. What are the main obstacles to a perfect merge?
> Historical names are in "SEHeadline" columns which holds huge variaty of form. Because SECompanyName and SEtickers are also not unique for each company we can not get certain primary key. 

### 3. What is special about CRSP names? Do you preprocess CRSP names before merging them? If so, what changes do you make?
> CRSP names are all with capital letters. To make fuzzy merge better in both dataset I made relevant columns lowercase.

### 4. Which tokens in the company names make your task challenging? How did you deal with them?
> Although most of the Headlines take a form "Q* 2*** COMNAM Earnings ***", there are different order of words. The most problematic headlines are ones which company name comes first and there various number of words between those names and year. Also some headlines don't contain name of the company.

### 5. What should be (theoretically) the number of records per firm’s fiscal year? What is the actual number of observations per firm’s fiscal year? Is this information helpful in identifying mismatches? Explain!
> In the case of ideal scenario, in every fiscal company would have Q1, Q2, Q3, Q4, Half Year, Full Year (Preliminary) observations of phone calls. In our dataset on annual average we have 3.36 observations within a fiscal year. Detailed information is given below.
> Knowing exactly how many observations to expect from each company would make it easy to find mismatches. In the case of excess matches we would be able to say that certain company mismatched. Data can be analyzed via looking for one company name under two same timeframe. 

### 6. What should be the relation between permco and the company name from the SEHeadline? More specifically, can one SEHeadline’s company name be correctly linked to multiple permcos (i.e., to more than one firm in CRSP)? Can one permco be correctly linked to multiple company names from SEHeadline?
> Permco represents a single company from a crsp dataset which includes all historical names of company. In SEHeadline column, company names change dynamically throughout time which means each permco can linked to several compnay names in SEHeadline.

### 7. What additional firm information would be helpful to check the validity of a match?
> To know which companies of SE data represented in the CRSP would make easy to eliminate irrelevant data.

In the case study pandas, numpy, re and thefuzz modules are used.

In [1]:
import pandas as pd 
import numpy as np
import re

Relevant dataframes are "crsp" which takes data from CRPnames.csv and "se" from SEmappingDAFA.csv.

In [2]:
crsp = pd.read_csv(r"/Users/javidrasulov/Desktop/Goethe/Season 2/Episode 1/DAFA/CaseStudy-1 SE - CRSP merge/Data/CRSPnames.csv")
se = pd.read_csv(r"/Users/javidrasulov/Desktop/Goethe/Season 2/Episode 1/DAFA/CaseStudy-1 SE - CRSP merge/Data/SEmappingsDAFA.csv")

Firstly I have checked for the most frequently used word in the "SEHeadline" column. It gave me initial understanding of the mainly used words to add regex.

In [88]:
# count is a list contains all strings from "SEHeadline" combained.
count = se['SEHeadline'].str.split().to_list()

In [89]:
# Loop convertes nested list to one flat list.
flat_list = []
for sublist in count:
    for item in sublist:
        flat_list.append(item)

Below, you can see most frequently used words in SEHeadlines. Some of them are relevant for regex. Quarters, earnings, call, conference words don't represent company names and should be removed.

In [87]:
pd.value_counts(np.array(flat_list))[0:10]

earnings      324901
call          310682
conference    184964
q3             76645
q2             76188
inc            75614
q1             75314
q4             70030
inc.           32569
corp           29343
dtype: int64

## TimeFrame analysis of Data

In this section, I tried to count the number of observation for each company in each year. As a result I got average number of observaions.

In [82]:
se_year = se

In [83]:
se_year

,SEid,SECompanyName,SEticker,SEHeadline,MergeComnam,SEHeadline_adj
0,12811556.0,1&1 Drillisch AG,DRI.DE,q3 2019 1&1 drillisch ag earnings call,NaN,1&1 drillisch ag
1,12353192.0,1&1 Drillisch AG,DRI.DE,q4 2018 1&1 drillisch ag earnings call,NaN,1&1 drillisch ag
2,12528454.0,1&1 Drillisch AG,DRI.DE,q1 2019 1&1 drillisch ag earnings call,NaN,1&1 drillisch ag
3,12623828.0,1&1 Drillisch AG,DRI.DE,q2 2019 1&1 drillisch ag earnings call,NaN,1&1 drillisch ag
4,13446819.0,1&1 Drillisch AG,DRIG.DE,q3 2020 1&1 drillisch ag earnings call,NaN,1&1 drillisch ag
...,...,...,...,...,...,...
327444,3390780.0,TDK Corp,6762.T,q2 2011 tdk earnings presentation and webcast ...,NaN,tdk
327445,2806790.0,TDK Corp,6762.T,full year 2010 tdk earnings presentation and w...,NaN,tdk
327446,3166510.0,TDK Corp,6762.T,q1 2011 tdk earnings presentation and webcast ...,NaN,tdk
327447,2545276.0,TDK Corp,6762.T,q3 2010 tdk earnings presentation (japanese),NaN,tdk


The "extract_year" function finds fiscal year of observation from the headlines. In the case of absence it gives null.

In [90]:
def extract_year(name):
    try:
        result = re.findall("2\d{3,}",name)[-1]
        return result
    except:
        pass

The "extract_year" function is applied via using multiprocess. I used pandarallel to avoid time consuming processing. Throughout notebook, in several cases I pursued similar approach.

In [91]:
pandarallel.initialize(progress_bar=True)

se_year["SEYear"] = se_year.parallel_apply(lambda row: extract_year(row['SEHeadline']),axis=1)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


The "extract_quarter" function finds quarter of observation from the headlines. In the case of absence it gives null.

In [92]:
def extract_quarter(name):
    try:
        result = re.findall("Q\d|First|Second|Third|Fourth|\dQ|Full Year|Half Year",name)[-1]
        return result
    except:
        pass

In [100]:
pandarallel.initialize(progress_bar=True)

se_year["SEQuarter"] = se_year.parallel_apply(lambda row: extract_quarter(row['SEHeadline']),axis=1)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


Only in 1,133 observations we are missing a year information. The effect is negligible.

In [102]:
se_year["SEYear"].isnull().sum()

1133

Below, code groups data with fiscal year and quarter.

In [95]:
se_grouped = se_year.groupby(["SECompanyName","SEYear"]).count()

In [97]:
print("The Number of Companies which had 5 calls within a year " + str(se_grouped[se_grouped["SEid"]==5].count()[1]))
print("The Number of Companies which had 4 calls within a year " + str(se_grouped[se_grouped["SEid"]==4].count()[1]))
print("The Number of Companies which had 3 calls within a year " + str(se_grouped[se_grouped["SEid"]==3].count()[1]))
print("The Number of Companies which had 2 calls within a year " + str(se_grouped[se_grouped["SEid"]==2].count()[1]))
print("The Number of Companies which had 1 calls within a year " + str(se_grouped[se_grouped["SEid"]==1].count()[1]))

The Number of Companies which had 5 calls within a year 1413
The Number of Companies which had 4 calls within a year 55458
The Number of Companies which had 3 calls within a year 12572
The Number of Companies which had 2 calls within a year 13829
The Number of Companies which had 1 calls within a year 11170


In [27]:
se_grouped["SEid"].describe()

count    96992.000000
mean         3.364360
std          1.446625
min          1.000000
25%          2.000000
50%          4.000000
75%          4.000000
max         36.000000
Name: SEid, dtype: float64

# Matching SEMapping and CRSP Dataframes

"namecut" function applies re.sub() formula. Via re.sub I tried to conditioning on "earning" word which occurs at the second half of the headlines and continues with irrelevant words. My regex removes "2\d{3,}" - two and following three digits which represent a year, "\w+.?quarter" - quarter, every word after earnings and analyst.

In [16]:
def namecut(name):
    name = re.sub("(?:(?=.*\bearning)^.*2\d{3,} |\w+.?quarter | ?earning\w+.*$|(?!.*\bearning)^.*2\d{3,} | analyst.*|conference.*|\w+.?quarter)","",name)
    return name

I used lowercase strings for precision.

In [5]:
se.SEHeadline = se.SEHeadline.str.lower()

I parallely applied namecut function on the "SEHeadline" column and created "SEHeadline_adj"

In [22]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

se["SEHeadline_adj"] = se.parallel_apply(lambda row: namecut(row['SEHeadline']),axis=1)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


Company name in crsp data is also lowercased.

In [23]:
crsp.COMNAM = crsp.COMNAM.str.lower()

In [24]:
from thefuzz import fuzz
from thefuzz import process

"match" function uses fuzzymerge to find relevant company name from crsp dataset.

In [51]:
def match(name):
    try:
        name = process.extractOne(name, crsp.COMNAM, score_cutoff = 85, scorer=fuzz.token_sort_ratio)[0]
        return name
    except:
        pass

Fuzzy merge applied on only uniqe values to avoid waste of time. The "uniques" dataframe contains unique values of adjusted headlines.

In [52]:
uniques = pd.DataFrame({ "SEHeadline_adj":se.SEHeadline_adj.unique()})

"match" function paralelly applied on unique values.

In [53]:
pandarallel.initialize(progress_bar=True)

uniques["COMNAM"] = uniques.parallel_apply(lambda row: match(row['SEHeadline_adj']),axis=1)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In the final part, "final_data" dataframe created by merging se and unique values. So, we get crsp company names for each of our observation.

In [60]:
final_data = pd.merge(se, uniques, how="left", left_on="SEHeadline_adj", right_on="SEHeadline_adj")

In [75]:
final_data = final_data.drop(columns=["MergeComnam"])

In [77]:
final_data = final_data.rename(columns={"COMNAM":"MergeComnam"})

Finally, we are calculating the number of matches which are notnull cells.

In [103]:
final_data.MergeComnam.notnull().sum()

205811